In [1]:
import pandas as pd
import os
import sys
from glob import glob
import numpy as np
import gc
from IPython.display import clear_output

main_dir = os.path.abspath(os.pardir)
sys.path.insert(0, main_dir)

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 100
pd.options.display.precision = 8
pd.options.display.float_format = '{:,.3f}'.format

In [2]:
data_path = os.path.join(main_dir, "data")+os.sep
intrm_trn_path = os.path.join(data_path, "interim", "train", "data")+os.sep
intrm_tst_path = os.path.join(data_path, "interim", "test", "data")+os.sep
prcsd_path = os.path.join(data_path, "processed")+os.sep
csv_files = pd.Series(glob(intrm_trn_path+"*.csv"))
csv_files = csv_files.append(pd.Series(glob(intrm_tst_path+"*.csv")), ignore_index=True)
csv_files

0     D:\Users\gjnet\code_projects\kaggle_ccs\data\interim\train\data\complete_train.csv
1           D:\Users\gjnet\code_projects\kaggle_ccs\data\interim\train\data\f0_train.csv
2             D:\Users\gjnet\code_projects\kaggle_ccs\data\interim\train\data\f0_val.csv
3           D:\Users\gjnet\code_projects\kaggle_ccs\data\interim\train\data\f1_train.csv
4             D:\Users\gjnet\code_projects\kaggle_ccs\data\interim\train\data\f1_val.csv
5           D:\Users\gjnet\code_projects\kaggle_ccs\data\interim\train\data\f2_train.csv
6             D:\Users\gjnet\code_projects\kaggle_ccs\data\interim\train\data\f2_val.csv
7           D:\Users\gjnet\code_projects\kaggle_ccs\data\interim\train\data\f3_train.csv
8             D:\Users\gjnet\code_projects\kaggle_ccs\data\interim\train\data\f3_val.csv
9           D:\Users\gjnet\code_projects\kaggle_ccs\data\interim\train\data\f4_train.csv
10            D:\Users\gjnet\code_projects\kaggle_ccs\data\interim\train\data\f4_val.csv
11      D:\Users\gjne

In [3]:
dfs = dict()
for file in csv_files:
    dfs[file.rsplit('\\', 1)[-1].replace(".csv", "")] = pd.read_csv(file, low_memory=False)
dfs.keys()

dict_keys(['complete_train', 'complete_test'])

In [4]:
complete_col_sum = dfs['complete_train'].sum(0)
cols_to_keep = complete_col_sum[complete_col_sum!=0].index
cols_to_keep

Index(['filename', ''Auto Color Correlogram0'', ''Auto Color Correlogram1'',
       ''Auto Color Correlogram2'', ''Auto Color Correlogram3'',
       ''Auto Color Correlogram48'', ''Auto Color Correlogram49'',
       ''Auto Color Correlogram50'', ''Auto Color Correlogram51'',
       ''Auto Color Correlogram52'',
       ...
       'PHOG621', 'PHOG622', 'PHOG623', 'PHOG624', 'PHOG625', 'PHOG626',
       'PHOG627', 'PHOG628', 'PHOG629', 'class'],
      dtype='object', length=1740)

In [5]:
def safe_downcast(series_in, num_type):
    series_out = pd.to_numeric(series_in, downcast=num_type)
    if not np.equal(series_in, series_out).all():
        series_out = series_in
    return series_out

In [6]:
for df in dfs.keys():
    print("Reducing dimensions on", df, end=" shape: ")
    dfs[df] = dfs[df][cols_to_keep]
    dfs[df]["type"] = dfs[df]["filename"].str.extract("^(trn|vld|tst)_", expand=False).rename("type").astype("category")
    dfs[df]["add"] = dfs[df]["filename"].str.extract("_(add)_", expand=False).rename("add").fillna("non_add").astype("category")
    dfs[df]["img_id"] = pd.to_numeric(dfs[df]["filename"].str.extract("_(\d+)_", expand=False).rename("img_id"), downcast="unsigned")

#     object_vnames = dfs[df].select_dtypes("O").columns
#     int_vnames = dfs[df].select_dtypes("integer").columns
#     float_vnames = dfs[df].select_dtypes("floating").columns
    
#     if len(object_vnames) > 0:
#         dfs[df][object_vnames] = dfs[df][object_vnames].astype("category")
#     if len(int_vnames) > 0:
#         dfs[df][int_vnames] = dfs[df][int_vnames].apply(safe_downcast, num_type="unsigned")
#     if len(float_vnames) > 0:
#         dfs[df][float_vnames] = dfs[df][float_vnames].apply(safe_downcast, num_type="float")
        
    print(dfs[df].shape)
gc.collect()
display(dfs[df].head())

Reducing dimensions on complete_train shape: (6730, 1743)
Reducing dimensions on complete_test shape: (4018, 1743)


,filename,'Auto Color Correlogram0','Auto Color Correlogram1','Auto Color Correlogram2','Auto Color Correlogram3','Auto Color Correlogram48','Auto Color Correlogram49','Auto Color Correlogram50','Auto Color Correlogram51','Auto Color Correlogram52','Auto Color Correlogram53','Auto Color Correlogram54','Auto Color Correlogram55','Auto Color Correlogram56','Auto Color Correlogram57','Auto Color Correlogram58','Auto Color Correlogram59','Auto Color Correlogram60','Auto Color Correlogram61','Auto Color Correlogram62','Auto Color Correlogram63','Spatial Pyramid of Local Binary Patterns0','Spatial Pyramid of Local Binary Patterns1','Spatial Pyramid of Local Binary Patterns2','Spatial Pyramid of Local Binary Patterns3','Spatial Pyramid of Local Binary Patterns4','Spatial Pyramid of Local Binary Patterns5','Spatial Pyramid of Local Binary Patterns6','Spatial Pyramid of Local Binary Patterns7','Spatial Pyramid of Local Binary Patterns8','Spatial Pyramid of Local Binary Patterns9','Spatial Pyramid of Local Binary Patterns10','Spatial Pyramid of Local Binary Patterns11','Spatial Pyramid of Local Binary Patterns12','Spatial Pyramid of Local Binary Patterns13','Spatial Pyramid of Local Binary Patterns14','Spatial Pyramid of Local Binary Patterns15','Spatial Pyramid of Local Binary Patterns16','Spatial Pyramid of Local Binary Patterns17','Spatial Pyramid of Local Binary Patterns18','Spatial Pyramid of Local Binary Patterns19','Spatial Pyramid of Local Binary Patterns20','Spatial Pyramid of Local Binary Patterns21','Spatial Pyramid of Local Binary Patterns22','Spatial Pyramid of Local Binary Patterns23','Spatial Pyramid of Local Binary Patterns24','Spatial Pyramid of Local Binary Patterns25','Spatial Pyramid of Local Binary Patterns26','Spatial Pyramid of Local Binary Patterns27','Spatial Pyramid of Local Binary Patterns29',...,PHOG584,PHOG585,PHOG586,PHOG587,PHOG588,PHOG589,PHOG590,PHOG591,PHOG592,PHOG593,PHOG594,PHOG595,PHOG596,PHOG597,PHOG598,PHOG599,PHOG600,PHOG601,PHOG602,PHOG603,PHOG604,PHOG605,PHOG606,PHOG607,PHOG608,PHOG609,PHOG610,PHOG611,PHOG612,PHOG613,PHOG614,PHOG615,PHOG616,PHOG617,PHOG618,PHOG619,PHOG620,PHOG621,PHOG622,PHOG623,PHOG624,PHOG625,PHOG626,PHOG627,PHOG628,PHOG629,class,type,add,img_id
0,tst_10000_nadd_.jpg,13,12,10,10,15,15,15,14,15,14,14,13,15,14,14,13,15,15,15,15,0,1,2,0,6,0,0,0,15,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,1,0,...,8,8,10,9,9,9,11,12,13,14,14,15,11,13,11,11,3,7,15,11,12,8,8,7,6,5,3,4,2,3,3,5,6,4,5,3,7,2,6,7,11,11,8,9,12,8,NaN,tst,non_add,10000
1,tst_10001_nadd_.jpg,7,6,6,5,15,14,14,14,15,14,14,13,15,14,14,13,15,15,15,14,0,0,0,0,2,0,0,0,15,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,...,1,2,2,1,4,10,12,15,9,8,12,11,8,8,6,5,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,5,9,15,12,6,4,3,3,2,1,1,2,2,NaN,tst,non_add,10001
2,tst_10002_nadd_.jpg,13,11,10,9,15,14,14,13,14,13,13,12,14,14,13,12,15,15,14,14,0,0,0,0,4,0,0,0,15,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,...,12,15,11,8,6,4,3,1,1,1,1,1,1,1,1,0,6,6,7,8,10,9,10,13,13,14,15,14,11,10,9,9,7,6,6,7,7,7,7,7,8,8,7,6,6,7,NaN,tst,non_add,10002
3,tst_10003_nadd_.jpg,10,8,6,5,15,15,15,14,14,13,13,12,14,13,12,11,15,14,13,13,1,2,3,0,9,0,0,0,15,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,3,0,...,9,10,8,7,9,11,13,13,14,15,13,12,9,8,9,8,10,7,8,9,11,13,15,13,13,13,12,10,9,9,10,12,11,11,10,10,11,10,11,12,11,12,9,9,9,9,NaN,tst,non_add,10003
4,tst_10004_nadd_.jpg,11,8,8,7,15,15,15,14,15,14,14,13,15,14,13,13,15,15,15,14,0,1,1,0,5,0,0,0,15,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,2,0,...,12,15,6,3,1,0,1,1,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,3,5,8,15,11,10,7,4,2,1,0,0,0,0,0,0,0,0,NaN,tst,non_add,10004


## File saving

In [7]:
for df in dfs.keys():
    dfs[df].to_parquet(prcsd_path+df+".gzip", engine="fastparquet", compression="gzip", index=False)

In [25]:
# from datetime import datetime
# START_TIME = datetime.now()

# # one of the following lines is uncommented before execution
# s = [np.float64(1), np.float32(1), np.float16(1), 1.0]

# for j, k in enumerate(s):
#     for i in range(10000000):
#         s[j] = (s[j] + 8) * s[j] % 2399232

#     print(s[j])
#     print('Runtime:', datetime.now() - START_TIME)
    

# 1340409.0
# Runtime: 0:00:11.860372
# 1340409.0
# Runtime: 0:00:23.970025
# 1340409.0
# Runtime: 0:00:35.694635
# 1340409.0
# Runtime: 0:00:39.258874

In [59]:
# def umap_trans(n_neighbors=15, min_dist=0.1, n_components=2, plot=False):
#     reducer = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components, metric="euclidean")
#     embedding = reducer.fit_transform(train_data[num_vnames])
#     print("Embedding shape", embedding.shape)
    
#     if plot:
#         sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})
#         plt.scatter(embedding[:, 0], embedding[:, 1], c=[sns.color_palette()[x] for x in train_data["class"].astype(int)])
#         plt.gca().set_aspect('equal', 'datalim')
#         plt.title('UMAP projection of the dataset', fontsize=24)
#     return reducer

In [60]:
# umap_trans(5, 0.1, 2, plot=True)

In [61]:
# max_feat = 100
# umap_reducer = dict()
# for vname_type in num_bd_vnames.keys():
#     if len(num_bd_vnames[vname_type]) > max_feat//len(num_bd_vnames.keys()):
#         print(vname_type, end=" ")
#         umap_reducer[vname_type] = umap_trans(5, 0.1, int(max_feat//len(num_bd_vnames.keys())))

In [62]:
# def apply_umap(data, num_vnames=num_bd_vnames, return_reducer=False):
#     y = data["class"].copy()
#     X = pd.DataFrame()
#     for vname_type in num_bd_vnames.keys():
#         if vname_type in umap_reducer.keys():
#             umap_reducer[vname_type] = umap_trans(5, 0.1, int(max_feat//len(num_bd_vnames.keys())))
#             X = pd.concat(
#                 [X, pd.DataFrame(umap_reducer[vname_type].transform(data[num_bd_vnames[vname_type]]), index=data.index).add_prefix(vname_type)], axis=1)
#         else:
#             X = pd.concat([X, data[num_bd_vnames[vname_type]]], axis=1)
#     if return_reducer:
#         return X, y, umap_reducer
#     else:
#         return X, y

In [63]:
# X_train, y_train, umap_reducer = apply_umap(train_data, return_reducer=True)
# X_test, y_test = apply_umap(test_data)